Processing Road Traffic Data

In [ ]:
from datetime import datetime, timedelta
import csv
city_date_time_volume={}
time_ranges = [(hour, hour+1) for hour in range(0, 24)]
time_range_counts = {f"{start_hour}-{end_hour}": 0 for start_hour, end_hour in time_ranges}
start_date = datetime(2022, 10, 6).date()
end_date = datetime(2023, 12, 10).date()
#Function to parse the date time 
def parse_datetime(year, month, day, hour, minute):
    return datetime(year, month, day, hour, minute)
#accessing the data 
with open("Road_traffic.csv", "r") as file:
    next(file)  
    for line in file:
        parts = line.strip().split(",")
        city = parts[1]
        year, month, day, hour, minute = map(int, parts[2:7])
        date_time = parse_datetime(year, month, day, hour, minute)
        date_key = date_time.date()
        if start_date <= date_key <= end_date:
            if city not in city_date_time_volume:
                city_date_time_volume[city] = {}
                city_date_time_volume[city][date_key]= {f"{start_hour}-{end_hour}": 0 for start_hour, end_hour in time_ranges}
            elif date_key not in city_date_time_volume[city]:
                city_date_time_volume[city][date_key] = {f"{start_hour}-{end_hour}": 0 for start_hour, end_hour in time_ranges}
            for start_hour, end_hour in time_ranges:
            #Checking if the datetime value falls within the current time range
                if start_hour <= int(parts[5]) < end_hour:
            #Incrementing the count for the current time range
                    city_date_time_volume[city][date_key][f"{start_hour}-{end_hour}"] += int(parts[7])
            #Break the loop since we've found the appropriate time range
                    break
#creating new csv file for adding the aggregated data
with open("aggregated_traffic_by_date_test.csv", "w", newline='') as output_file:
    writer = csv.writer(output_file)
    row = ["city","Date"] + [f"{i}" for i in time_range_counts.keys() ]
    writer.writerow(row)
    for city, date_time_counts in city_date_time_volume.items():
        for date, time_counts in date_time_counts.items():
            row = [city, date] + [count for count in time_counts.values()]
            writer.writerow(row)

        

Accessing and processing Air taffic data

In [ ]:
import paramiko
import datetime
import os
import zipfile
import csv
import json
# function to extract the data from the file and adding them in to the excel sheet.
def ETL(filename):
    time_ranges = [(hour, hour+1) for hour in range(0, 24)]
    time_range_counts = {f"{start_hour}-{end_hour}": 0 for start_hour, end_hour in time_ranges}
    #accessing each file
    with open(filename, 'r') as file:
    # Reading each line of the file
        for line in file:
            try:
            # Parsing the line into a dictionary
                data = json.loads(line)
                data["dt"] = data["dt"].split(".")
                dt=datetime.datetime.strptime(data["dt"][0],"%Y-%m-%d %H:%M:%S")
                date= dt.date()
                for start_hour, end_hour in time_ranges:
        # Checking if the datetime value falls within the current time range
                    if start_hour <= dt.hour < end_hour:
            # Increment the count for the current time range
                        time_range_counts[f"{start_hour}-{end_hour}"] += 1
            # Breaking the loop since we've found the appropriate time range
                        break
           
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line.strip()}. Error: {e}")
                continue
        
    with open('air_traffic_counts.csv', 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        row = ["Air Traffic", date] + [time_range_counts.get(time_range, 0) for time_range in time_range_counts.keys()]
        csv_writer.writerow(row)

# Function to download and process the zip file
def download_and_process_zip_file(sftp, date):
    filename1 = f'adsblog_ct0.txt.{date.strftime("%Y%m%d")}00.zip'
    # construct the filename based on the date pattern
    filename = f'adsblog_ct0.txt.{date.strftime("%Y%m%d")}00'
    remote_path = f'server file path'

    # downloading the zip file
    local_zip_path = f"local file path to download the data"  # Path where you want to save the downloaded zip file locally
    sftp.get(remote_path, local_zip_path)
    print(f"Downloaded {filename}.zip")

    # unziping the downloaded file
    extracted_folder_path = f"file path to exctract the dwnloaded file"
    Deleting_extracted_folder_path = f"file path to delete extracted file path"   # Path where you want to extract the contents of the zip file
    os.makedirs(extracted_folder_path, exist_ok=True)
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder_path)
    print(f"Extracted {filename}.zip")
    ETL(filename)
    print("Data has been added to the excel sheet")
    # deleting the downloaded zip file and extracted file
    os.remove(local_zip_path)
    print(f"Deleted {filename}.zip")
    os.remove(Deleting_extracted_folder_path)
    print(f"Deleted {filename}")

# SSH connection parameters
host = "Provide the host name"
port = "port number in int"
username = 'username'
password = '*********'

# Start and end dates
start_date = datetime.date(2022, 10, 9)
end_date = datetime.date(2023, 12, 10)

# creating an SSH client
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # Automatically add host keys

try:
    # connecting to the SSH server
    ssh_client.connect(hostname=host, port=port, username=username, password=password)
    print("Connected to the server.")

    # opening an SFTP session
    sftp = ssh_client.open_sftp()
    print("SFTP session opened.")

    # iterating over the dates
    current_date = start_date
    while current_date <= end_date:
        download_and_process_zip_file(sftp, current_date)
        current_date += datetime.timedelta(days=1)

finally:
    #Closing connection
    if 'sftp' in locals():
        sftp.close()
        print("SFTP session closed.")
    ssh_client.close()
    print("SSH connection closed.")
#it would be better to print the step of the process at the end of each iteration


code for Volume Over Month-Years in 2023

In [ ]:
import csv
import matplotlib.pyplot as plt

# Function to read data from CSV file
def read_csv(input_file):
    data = []
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

# Function to plot the graph
def plot_graph(data):
    # Filter data for the year 2023
    data_2023 = [entry for entry in data if entry['Date'].startswith('2023')]

    # Group data by city and month-year
    city_data = {}
    for entry in data_2023:
        city = entry['City']
        date = entry['Date']
        month_year = date.split('-')[1] + '-' + date.split('-')[0]  # Swap month and year
        if city not in city_data:
            city_data[city] = {}
        if month_year not in city_data[city]:
            city_data[city][month_year] = []
        city_data[city][month_year].append(float(entry['Volume']))

    # Sort month-year values
    all_month_years = sorted(list(set(month_year for city_info in city_data.values() for month_year in city_info.keys())))

    # Plot data for each city
    for city, city_info in city_data.items():
        volumes = [sum(city_info.get(month_year, [0])) for month_year in all_month_years]
        plt.plot(all_month_years, volumes, label=city)

    # Customize the plot
    plt.xlabel('Month-Year')
    plt.ylabel('Volume')
    plt.title('Volume Over Month-Years in 2023 (Grouped by City)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # Show the plot
    plt.show()

# Example usage
input_file = 'merged_data.csv'  # Replace with your CSV file
data = read_csv(input_file)
plot_graph(data)


code for Scaled traffic Volume Over Month-Years in 2023

In [ ]:
import csv
import matplotlib.pyplot as plt

# Function to read data from CSV file
def read_csv(input_file):
    data = []
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

# Function to plot the graph
def plot_graph(data):
    # Filter data for the year 2023
    data_2023 = [entry for entry in data if entry['Date'].startswith('2023')]

    # Group data by city and month-year
    city_data = {}
    for entry in data_2023:
        city = entry['City']
        date = entry['Date']
        month_year = date.split('-')[1] + '-' + date.split('-')[0]  # Swap month and year
        if city not in city_data:
            city_data[city] = {}
        if month_year not in city_data[city]:
            city_data[city][month_year] = []
        city_data[city][month_year].append(float(entry['Scaled Volume']))

    # Sort month-year values
    all_month_years = sorted(list(set(month_year for city_info in city_data.values() for month_year in city_info.keys())))

    # Plot data for each city
    for city, city_info in city_data.items():
        volumes = [sum(city_info.get(month_year, [0])) for month_year in all_month_years]
        plt.plot(all_month_years, volumes, label=city)

    # Customize the plot
    plt.xlabel('Month-Year')
    plt.ylabel('Scaled traffic Volume')
    plt.title('Scaled traffic Volume Over Month-Years in 2023 (Grouped by City)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # Show the plot
    plt.show()

# Example usage
input_file = 'merged_data.csv'  # Replace with your CSV file
data = read_csv(input_file)
plot_graph(data)


code for Volume Over Month-Years in 2023 (Grouped by Traffic Type)

In [ ]:
import csv
import matplotlib.pyplot as plt

# Function to read data from CSV file
def read_csv(input_file):
    data = []
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

# Function to plot the graph
def plot_graph(data):
    # Filter data for the year 2023
    data_2023 = [entry for entry in data if entry['Date'].startswith('2023')]

    # Group data by road traffic and air traffic
    road_traffic_data = {}
    air_traffic_data = {}
    for entry in data_2023:
        city = entry['City']
        month_year = entry['Date'].split('-')[1] + '-' + entry['Date'].split('-')[0]  # Swap month and year
        if city == 'Air Traffic':
            if month_year not in air_traffic_data:
                air_traffic_data[month_year] = []
            air_traffic_data[month_year].append(float(entry['Volume']))
        else:
            if month_year not in road_traffic_data:
                road_traffic_data[month_year] = []
            road_traffic_data[month_year].append(float(entry['Volume']))

    # Sort month-year values
    all_month_years = sorted(list(set(month_year for month_year in road_traffic_data.keys()).union(set(month_year for month_year in air_traffic_data.keys()))))

    # Plot data for road traffic
    road_traffic_volumes = [sum(road_traffic_data.get(month_year, [0])) for month_year in all_month_years]
    plt.plot(all_month_years, road_traffic_volumes, label='Road Traffic')

    # Plot data for air traffic
    air_traffic_volumes = [sum(air_traffic_data.get(month_year, [0])) for month_year in all_month_years]
    plt.plot(all_month_years, air_traffic_volumes, label='Air Traffic')

    # Customize the plot
    plt.xlabel('Month-Year')
    plt.ylabel('Volume')
    plt.title('Volume Over Month-Years in 2023 (Grouped by Traffic Type)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # Show the plot
    plt.show()

# Example usage
input_file = 'merged_data.csv'  # Replace with your CSV file
data = read_csv(input_file)
plot_graph(data)


code for Volume Over Days in 2023 (Grouped by Traffic Type)

In [ ]:
import csv
import matplotlib.pyplot as plt

# Function to read data from CSV file
def read_csv(input_file):
    data = []
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

# Function to plot the graph
def plot_graph(data):
    # Filter data for the year 2023
    data_2023 = [entry for entry in data if entry['Date'].startswith('2023')]

    # Group data by traffic type and day
    traffic_data = {'Road Traffic': {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 0, 'Sunday': 0},
                    'Air Traffic': {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 0, 'Sunday': 0}}
    for entry in data_2023:
        city = entry['City']
        day = entry['Day']
        volume = float(entry['Volume'])
        if city == 'Air Traffic':
            traffic_data['Air Traffic'][day] += volume
        else:
            traffic_data['Road Traffic'][day] += volume

    # Plot data for road traffic
    road_traffic_days = list(traffic_data['Road Traffic'].keys())
    road_traffic_volumes = [traffic_data['Road Traffic'][day] for day in road_traffic_days]
    plt.plot(road_traffic_days, road_traffic_volumes, label='Road Traffic')

    # Plot data for air traffic
    air_traffic_days = list(traffic_data['Air Traffic'].keys())
    air_traffic_volumes = [traffic_data['Air Traffic'][day] for day in air_traffic_days]
    plt.plot(air_traffic_days, air_traffic_volumes, label='Air Traffic')

    # Customize the plot
    plt.xlabel('Day')
    plt.ylabel('Volume')
    plt.title('Volume Over Days in 2023 (Grouped by Traffic Type)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # Show the plot
    plt.show()

# Example usage
input_file = 'merged_data.csv'  # Replace with your CSV file
data = read_csv(input_file)
plot_graph(data)


code for Volume Over Time Ranges in 2023 (Grouped by Traffic Type)

In [ ]:
import csv
import matplotlib.pyplot as plt

# Function to read data from CSV file
def read_csv(input_file):
    data = []
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

# Function to plot the graph
def plot_graph(data):
    # Filter data for the year 2023
    data_2023 = [entry for entry in data if entry['Date'].startswith('2023')]

    # Group data by traffic type and time range
    traffic_data = {'Road Traffic': {}, 'Air Traffic': {}}
    for entry in data_2023:
        city = entry['City']
        time_range = entry['Time_range']
        volume = float(entry['Volume'])
        if city == 'Air Traffic':
            if time_range not in traffic_data['Air Traffic']:
                traffic_data['Air Traffic'][time_range] = 0
            traffic_data['Air Traffic'][time_range] += volume
        else:
            if time_range not in traffic_data['Road Traffic']:
                traffic_data['Road Traffic'][time_range] = 0
            traffic_data['Road Traffic'][time_range] += volume

    # Plot data for road traffic
    road_traffic_time_ranges = list(traffic_data['Road Traffic'].keys())
    road_traffic_volumes = [traffic_data['Road Traffic'][time_range] for time_range in road_traffic_time_ranges]
    plt.plot(road_traffic_time_ranges, road_traffic_volumes, label='Road Traffic')

    # Plot data for air traffic
    air_traffic_time_ranges = list(traffic_data['Air Traffic'].keys())
    air_traffic_volumes = [traffic_data['Air Traffic'][time_range] for time_range in air_traffic_time_ranges]
    plt.plot(air_traffic_time_ranges, air_traffic_volumes, label='Air Traffic')

    # Customize the plot
    plt.xlabel('Time Range')
    plt.ylabel('Volume')
    plt.title('Volume Over Time Ranges in 2023 (Grouped by Traffic Type)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # Show the plot
    plt.show()

# Example usage
input_file = 'merged_data.csv'  # Replace with your CSV file
data = read_csv(input_file)
plot_graph(data)


Code for adding the data in to phpMyadmin

In [2]:
import pymysql,csv,time,datetime
with open('merged_data.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    
start = time.time()
#connecting to the server
conn = pymysql.connect(host='mysql.clarksonmsda.org', port =3306, user ='ia626',
                       passwd='ia626clarkson', db='ia626',autocommit = True)
cur = conn.cursor(pymysql.cursors.DictCursor)
#droping table if exist
cur.execute("DROP TABLE IF EXISTS `padakas_roadtraffic_vs_airtraffic`;")
#creating the new table
sql = '''CREATE TABLE IF NOT EXISTS `padakas_roadtraffic_vs_airtraffic` (
  `id` int NOT NULL AUTO_INCREMENT,
  `City` varchar(50) NOT NULL,
  `Date` date NOT NULL,
  `Day` varchar(15) NOT NULL,
  `Time_range` varchar(10) NOT NULL,
  `Voulme` int NULL,
  PRIMARY KEY(`id`)
) ENGINE=MyISAM DEFAULT CHARSET=latin1 AUTO_INCREMENT = 1;'''
cur.execute(sql)
#insert statement to add data.
sql= '''INSERT INTO `padakas_roadtraffic_vs_airtraffic`(`City`,`Date`,`Day`,`Time_range`,`Voulme`) VALUES(%s,%s,%s,%s,%s);'''
tokens=[]
blocksize = 10000
for row in data:
    tokens.append([row['City'],row['Date'],row['Day'],row['Time_range'],row['Volume']])
    if len(tokens) >= blocksize:
        cur.executemany(sql,tokens)
        tokens = []

if len(tokens)>0:
    cur.executemany(sql,tokens)

print("Time taken for exectution: " + str(time.time()-start))   

Time taken for exectution: 1.6386339664459229
